# Rohaan Advani - rna3535 - N19682109

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import re
import pyspark
from pyspark.sql.functions import col, udf, split, concat_ws, to_timestamp, weekofyear, sum, when, hour, dayofweek, collect_list, count, explode, lower, regexp_replace
from pyspark.sql.types import TimestampType, FloatType, ArrayType, StringType
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from math import radians, sin, cos, sqrt, atan2
import logging

In [2]:
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041') 
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/17 16:55:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1. 15 points

Datafile: Bakery.csv

Solve: Show the total number bought by item, per week, for the time period
between 11AM and 1PM

For example (these are made up numbers….)

Item day qty

Bread, 2016-10-30, 102

In [3]:
bakery = spark.read.option("inferSchema", "true").option("header", "true").csv("shared/hw2/Bakery.csv")
bakery.limit(10).toPandas()

,Date,Time,Transaction,Item
0,2016-10-30,2024-03-17 09:58:11,1,Bread
1,2016-10-30,2024-03-17 10:05:34,2,Scandinavian
2,2016-10-30,2024-03-17 10:05:34,2,Scandinavian
3,2016-10-30,2024-03-17 10:07:57,3,Hot chocolate
4,2016-10-30,2024-03-17 10:07:57,3,Jam
5,2016-10-30,2024-03-17 10:07:57,3,Cookies
6,2016-10-30,2024-03-17 10:08:41,4,Muffin
7,2016-10-30,2024-03-17 10:13:03,5,Coffee
8,2016-10-30,2024-03-17 10:13:03,5,Pastry
9,2016-10-30,2024-03-17 10:13:03,5,Bread


In [4]:
# Remove today's date from time column
bakery = bakery.withColumn("time", split(col("Time"), " ")[1])
bakery.limit(10).toPandas()

,Date,time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
2,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam
5,2016-10-30,10:07:57,3,Cookies
6,2016-10-30,10:08:41,4,Muffin
7,2016-10-30,10:13:03,5,Coffee
8,2016-10-30,10:13:03,5,Pastry
9,2016-10-30,10:13:03,5,Bread


In [5]:
# Filter out transactions between 11 AM and 1 PM
bakery_filtered = bakery.filter((col("time") >= "11:00:00") & (col("time") <= "13:00:00"))
bakery_filtered.limit(10).toPandas()

,Date,time,Transaction,Item
0,2016-10-30,11:02:19,27,Scandinavian
1,2016-10-30,11:03:24,28,Coffee
2,2016-10-30,11:03:24,28,Bread
3,2016-10-30,11:05:30,29,Bread
4,2016-10-30,11:05:30,29,Jam
5,2016-10-30,11:05:30,29,NONE
6,2016-10-30,11:07:19,30,Bread
7,2016-10-30,11:12:56,31,Basket
8,2016-10-30,11:16:15,32,Scandinavian
9,2016-10-30,11:16:15,32,Muffin


In [6]:
# Convert date and time columns to timestamp
bakery_filtered = bakery_filtered.withColumn("timestamp_str", concat_ws(" ", col("Date"), col("time")))
bakery_filtered = bakery_filtered.withColumn("timestamp", to_timestamp(col("timestamp_str"), "yyyy-MM-dd HH:mm:ss"))
bakery_filtered = bakery_filtered.drop("timestamp_str")
bakery_filtered = bakery_filtered.drop("Date")
bakery_filtered = bakery_filtered.drop("time")
bakery_filtered.limit(10).toPandas()

,Transaction,Item,timestamp
0,27,Scandinavian,2016-10-30 11:02:19
1,28,Coffee,2016-10-30 11:03:24
2,28,Bread,2016-10-30 11:03:24
3,29,Bread,2016-10-30 11:05:30
4,29,Jam,2016-10-30 11:05:30
5,29,NONE,2016-10-30 11:05:30
6,30,Bread,2016-10-30 11:07:19
7,31,Basket,2016-10-30 11:12:56
8,32,Scandinavian,2016-10-30 11:16:15
9,32,Muffin,2016-10-30 11:16:15


In [7]:
result_q1 = bakery_filtered.groupBy("Item", weekofyear("timestamp").alias("week")).agg(sum("Transaction").alias("qty"))
result_q1.toPandas()

,Item,week,qty
0,Tartine,44,4511
1,Sandwich,45,7502
2,Truffles,52,3884
3,Salad,3,4927
4,Cookies,8,42645
...,...,...,...
719,Brownie,51,10908
720,Jam,6,6267
721,Tacos/Fajita,14,19185
722,Basket,43,31


# 2. 20 points

Show the top 3 (by qty) items bought grouped by Daypart, by DayType.

Must be in the format shown

For example (not necessarily the right numbers….)

Morning, (bread, pastry, Muffin), Weekend

In [8]:
# Add DayType and DayPart Columns
bakery = bakery.withColumn('DayType', when((dayofweek('Date') >= 2) & (dayofweek('Date') <= 6), 'Weekday').otherwise('Weekend')) \
      .withColumn('DayPart', when((hour('time') >= 4) & (hour('time') < 12), 'Morning')
                                .when((hour('time') >= 12) & (hour('time') < 16), 'Afternoon')
                                .when((hour('time') >= 16) & (hour('time') < 20), 'Evening')
                                .otherwise('Night'))
bakery.limit(10).toPandas()

,Date,time,Transaction,Item,DayType,DayPart
0,2016-10-30,09:58:11,1,Bread,Weekend,Morning
1,2016-10-30,10:05:34,2,Scandinavian,Weekend,Morning
2,2016-10-30,10:05:34,2,Scandinavian,Weekend,Morning
3,2016-10-30,10:07:57,3,Hot chocolate,Weekend,Morning
4,2016-10-30,10:07:57,3,Jam,Weekend,Morning
5,2016-10-30,10:07:57,3,Cookies,Weekend,Morning
6,2016-10-30,10:08:41,4,Muffin,Weekend,Morning
7,2016-10-30,10:13:03,5,Coffee,Weekend,Morning
8,2016-10-30,10:13:03,5,Pastry,Weekend,Morning
9,2016-10-30,10:13:03,5,Bread,Weekend,Morning


In [9]:
result_q2 = bakery.groupBy('DayPart', 'DayType', 'Item').agg(sum('Transaction').alias('qty')).orderBy(col('qty').desc())
result_q2.limit(10).toPandas()

,DayPart,DayType,Item,qty
0,Morning,Weekday,Coffee,7903336
1,Afternoon,Weekday,Coffee,7866868
2,Afternoon,Weekend,Coffee,4787274
3,Morning,Weekday,Bread,4695939
4,Morning,Weekend,Coffee,4419327
5,Afternoon,Weekday,Bread,4411892
6,Morning,Weekend,Bread,2971599
7,Afternoon,Weekend,Bread,2798884
8,Afternoon,Weekday,Tea,2615817
9,Afternoon,Weekday,Sandwich,2267671


In [10]:
result_q2 = result_q2.groupBy('DayPart', 'DayType').agg(collect_list('Item').alias('Items')).limit(3)
result_q2.toPandas()

,DayPart,DayType,Items
0,Night,Weekend,"[Vegan Feast, Postcard, Tshirt, Pintxos, Scand..."
1,Night,Weekday,"[Valentine's card, Juice, Mineral water, Coffe..."
2,Morning,Weekend,"[Coffee, Bread, Pastry, Medialuna, NONE, Cake,..."


# 3. 15 Points

Dataset: Restaurants_in_Durham_County_NC.json

Solve: How many food service establishments have “SUBWAY” in their name?

Example:
25

In [11]:
restaurants = spark.read.json("shared/hw2/Restaurants_in_Durham_County_NC.json")
restaurants.limit(10).toPandas()

,datasetid,fields,geometry,record_timestamp,recordid
0,restaurants-data,"(None, Full-Service Restaurant, [35.9207272, -...","([-78.9573299, 35.9207272], Point)",2017-07-13T09:15:31-04:00,1644654b953d1802c3c941211f61be1f727b2951
1,restaurants-data,"(None, Nursing Home, [36.0467802, -78.8895483]...","([-78.8895483, 36.0467802], Point)",2017-07-13T09:15:31-04:00,93573dbf8c9e799d82c459e47de0f40a2faa47bb
2,restaurants-data,"(None, Fast Food Restaurant, [35.9182655, -78....","([-78.9593263, 35.9182655], Point)",2017-07-13T09:15:31-04:00,0d274200c7cef50d05dd633bc5da896a69e1bb99
3,restaurants-data,"(None, Full-Service Restaurant, [36.0183378, -...","([-78.9060312, 36.0183378], Point)",2017-07-13T09:15:31-04:00,cf3e0b175a6ebad2a9aa8edb4b3985c6391ed31d
4,restaurants-data,"(None, None, [36.0556347, -78.9135175], None, ...","([-78.9135175, 36.0556347], Point)",2017-07-13T09:15:31-04:00,e796570677f7c39cc90d4870f192a9beb928560a
5,restaurants-data,"(None, None, [35.984012, -78.8077969], None, 6...","([-78.8077969, 35.984012], Point)",2017-07-13T09:15:31-04:00,90cdb7722ea7d4ffde931d1b9c45fec2ad97378e
6,restaurants-data,"(None, None, [35.9857413, -78.8865863], None, ...","([-78.8865863, 35.9857413], Point)",2017-07-13T09:15:31-04:00,a777677a2690466efa03b77e411566eecd76d26f
7,restaurants-data,"(None, None, [35.913596, -78.9602128], None, 5...","([-78.9602128, 35.913596], Point)",2017-07-13T09:15:31-04:00,f8e7a2243f9edb2cd5d08c13add936bd43eca09c
8,restaurants-data,"(None, None, [36.0111429, -78.953045], None, 5...","([-78.953045, 36.0111429], Point)",2017-07-13T09:15:31-04:00,b13c50f485bd92b042fd065523c58e5480531f9b
9,restaurants-data,"(None, None, [36.0184133, -78.947422], None, 5...","([-78.947422, 36.0184133], Point)",2017-07-13T09:15:31-04:00,f71fd314be631c0fe1226cf4485385d992ea45b6


In [12]:
restaurants = restaurants.select(col("fields.*"))
restaurants.limit(10).toPandas()

,closing_date,est_group_desc,geolocation,hours_of_operation,id,insp_freq,opening_date,premise_address1,premise_address2,premise_city,...,premise_zip,risk,rpt_area_desc,seats,sewage,smoking_allowed,status,transitional_type_desc,type_description,water
0,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,27707,4,Food Service,60.0,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community
1,None,Nursing Home,"[36.0467802, -78.8895483]",None,58123,4,2003-10-15,4434 BEN FRANKLIN BLVD,None,DURHAM,...,27704,4,Food Service,350.0,3 - Municipal/Community,NO,ACTIVE,FOOD,16 - Institutional Food Service,5 - Municipal/Community
2,None,Fast Food Restaurant,"[35.9182655, -78.9593263]",None,70266,2,2009-07-09,1125 W. NC HWY 54 SUITE 806,None,DURHAM,...,27707,2,Food Service,7.0,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community
3,None,Full-Service Restaurant,"[36.0183378, -78.9060312]",None,97837,2,2012-01-09,1542 N GREGSON ST,None,DURHAM,...,27701,2,Food Service,100.0,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community
4,None,None,"[36.0556347, -78.9135175]",None,60690,1,2008-06-02,909 GARCIA ST,None,DURHAM,...,27704,0,Residential Care,6.0,3 - Municipal/Community,N/A,ACTIVE,N/A,43 - Residential Care,5 - Municipal/Community
5,None,None,"[35.984012, -78.8077969]",None,60686,1,2008-06-02,4 KIMBROUGH COURT,None,DURHAM,...,27703,0,Residential Care,0.0,3 - Municipal/Community,N/A,ACTIVE,N/A,43 - Residential Care,5 - Municipal/Community
6,None,None,"[35.9857413, -78.8865863]",None,85252,2,2010-08-23,1309 ANGIER AVE.,None,DURHAM,...,27701,0,Day Care,NaN,3 - Municipal/Community,NO,ACTIVE,N/A,42 - Child Care,5 - Municipal/Community
7,None,None,"[35.913596, -78.9602128]",None,59120,2,1996-04-02,7304 CALIBRE PARK DR,None,DURHAM,...,27707,0,Swimming Pools,0.0,3 - Municipal/Community,N/A,ACTIVE,N/A,53 - Year-Round Swimming Pool,5 - Municipal/Community
8,None,None,"[36.0111429, -78.953045]",None,59124,2,1996-04-25,2800 CAMPUS WALK AVE,None,DURHAM,...,27705,0,Swimming Pools,0.0,3 - Municipal/Community,N/A,ACTIVE,N/A,53 - Year-Round Swimming Pool,5 - Municipal/Community
9,None,None,"[36.0184133, -78.947422]",None,59263,2,2000-05-30,3419 HILLSBOROUGH RD,None,DURHAM,...,27705,0,Swimming Pools,0.0,3 - Municipal/Community,N/A,ACTIVE,N/A,55 - Year-Round Spa,5 - Municipal/Community


In [13]:
# Select the premise_name column
restaurant_names = restaurants.select(col("premise_name"))
restaurant_names.limit(10).toPandas()

,premise_name
0,WEST 94TH ST PUB
1,BROOKDALE DURHAM IFS
2,SMOOTHIE KING
3,HAMPTON INN & SUITES
4,BETTER LIVING CONCEPTS OF DURHAM
5,ADVENTURE HOUSE
6,ANOTHER BEAUTIFUL BEGINNINGS
7,BRIDGES AT SOUTHPOINT APTS
8,SHEARTON INN UNIVERSITY
9,SPA HEALTH CLUB


In [14]:
result_q3 = restaurant_names.filter(col("premise_name").contains("SUBWAY")).agg(count("*").alias("subway_count")).collect()[0]["subway_count"]
result_q3

25

# 4. 35 Points

Dataset: populationbycountry19802010millions.csv

Solve: For each year, show the region with the biggest percentage increase in population, year over year. Ignore year 1980.

For example, the year over year for North America in 1981 is:

(324.44694 - 320.27638) / 320.2763 = 1.30%

Example:

1981, North America, 9.30% <- assuming North America was the max increase

1982, Aruba, … <- assuming Aruba was max decrease that year

In [15]:
population = spark.read.option("inferSchema", "true").option("header", "true").csv("shared/hw2/populationbycountry19802010millions.csv")
population.limit(10).toPandas()

24/03/17 16:55:47 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/03/17 16:55:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
 Schema: _c0, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv


,_c0,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010
0,North America,320.27638,324.44694,328.62014,332.72487,336.72143,340.74811,344.89548,349.07829,353.2939,...,417.83236,422.05268,426.06238,430.26938,434.47232,438.82964,443.3473,447.67394,451.83698,456.59331
1,Bermuda,0.05473,0.05491,0.05517,0.05551,0.05585,0.05618,0.05651,0.05683,0.05717,...,0.06361,0.06418,0.06476,0.06534,0.06591,0.06644,0.06692,0.06739,0.06784,0.06827
2,Canada,24.5933,24.9,25.2019,25.4563,25.7018,25.9416,26.2038,26.5497,26.8948,...,31.37674,31.64096,31.88931,32.13476,32.38638,32.65668,32.93596,33.2127,33.48721,33.75974
3,Greenland,0.05021,0.05103,0.05166,0.05211,0.05263,0.05315,0.05364,0.0541,0.05485,...,0.05713,0.05736,0.05754,0.0577,0.05778,0.05764,0.05753,0.05756,0.0576,0.05764
4,Mexico,68.34748,69.96926,71.6409,73.36288,75.08014,76.76723,78.44243,80.12249,81.78182,...,101.24696,102.47993,103.71806,104.95959,106.2029,107.44953,108.70089,109.9554,111.21179,112.46886
5,Saint Pierre and Miquelon,0.00599,0.00601,0.00605,0.00607,0.00611,0.00616,0.00621,0.00625,0.00628,...,0.00637,0.00633,0.00629,0.00625,0.0062,0.00615,0.0061,0.00605,0.006,0.00594
6,United States,227.22468,229.46571,231.66446,233.79199,235.8249,237.9238,240.13289,242.28892,244.49898,...,285.08156,287.80391,290.32642,293.04574,295.75315,298.59321,301.5799,304.37485,307.00655,310.23286
7,Central & South America,293.05856,299.43033,305.95253,312.51136,318.87955,325.22704,331.82291,338.59859,345.44544,...,427.24012,433.05116,438.97976,445.01525,451.05504,457.01699,462.89157,468.73872,474.53897,480.01228
8,Antarctica,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,Antigua and Barbuda,0.06855,0.06826,0.06801,0.06562,0.06447,0.0644,0.0644,0.06442,0.06443,...,0.07673,0.07794,0.07907,0.08019,0.08128,0.08234,0.08343,0.08452,0.08563,0.08675


In [16]:
# Calculate year-over-year percentage increase for each country
percent_inc_cols = []
for year in range(1981, 2011):
    percent_inc_cols.append(f"{year}_%_inc")
    population = population.withColumn(f"{year}_%_inc", (F.col(str(year)) - F.col(str(year - 1))) / F.col(str(year - 1)) * 100)

population = population.na.fill(0)
population.limit(10).toPandas()

24/03/17 16:55:52 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
 Schema: _c0, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv


,_c0,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2001_%_inc,2002_%_inc,2003_%_inc,2004_%_inc,2005_%_inc,2006_%_inc,2007_%_inc,2008_%_inc,2009_%_inc,2010_%_inc
0,North America,320.27638,324.44694,328.62014,332.72487,336.72143,340.74811,344.89548,349.07829,353.2939,...,1.090635,1.010051,0.950047,0.987414,0.976816,1.002899,1.029479,0.975903,0.929927,1.052665
1,Bermuda,0.05473,0.05491,0.05517,0.05551,0.05585,0.05618,0.05651,0.05683,0.05717,...,0.872185,0.896086,0.903708,0.895615,0.872360,0.804127,0.722456,0.702331,0.667755,0.633844
2,Canada,24.5933,24.9,25.2019,25.4563,25.7018,25.9416,26.2038,26.5497,26.8948,...,0.891267,0.842089,0.784900,0.769694,0.783015,0.834610,0.855200,0.840237,0.826521,0.813833
3,Greenland,0.05021,0.05103,0.05166,0.05211,0.05263,0.05315,0.05364,0.0541,0.05485,...,0.421867,0.402591,0.313808,0.278067,0.138648,-0.242298,-0.190840,0.052147,0.069493,0.069444
4,Mexico,68.34748,69.96926,71.6409,73.36288,75.08014,76.76723,78.44243,80.12249,81.78182,...,1.321310,1.217785,1.208168,1.197024,1.184561,1.173819,1.164603,1.154094,1.142636,1.130339
5,Saint Pierre and Miquelon,0.00599,0.00601,0.00605,0.00607,0.00611,0.00616,0.00621,0.00625,0.00628,...,-0.624025,-0.627943,-0.631912,-0.635930,-0.800000,-0.806452,-0.813008,-0.819672,-0.826446,-1.000000
6,United States,227.22468,229.46571,231.66446,233.79199,235.8249,237.9238,240.13289,242.28892,244.49898,...,1.031144,0.954937,0.876468,0.936642,0.923886,0.960281,1.000254,0.926769,0.864625,1.050893
7,Central & South America,293.05856,299.43033,305.95253,312.51136,318.87955,325.22704,331.82291,338.59859,345.44544,...,1.372919,1.360134,1.369030,1.374890,1.357210,1.321779,1.285418,1.263179,1.237416,1.153395
8,Antarctica,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Antigua and Barbuda,0.06855,0.06826,0.06801,0.06562,0.06447,0.0644,0.0644,0.06442,0.06443,...,1.831453,1.576958,1.449833,1.416466,1.359272,1.304134,1.323779,1.306484,1.313299,1.307953


In [17]:
columns_to_drop = population.columns[1:32]
population = population.drop(*columns_to_drop)
population.limit(10).toPandas()

24/03/17 16:55:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
 Schema: _c0, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv


,_c0,1981_%_inc,1982_%_inc,1983_%_inc,1984_%_inc,1985_%_inc,1986_%_inc,1987_%_inc,1988_%_inc,1989_%_inc,...,2001_%_inc,2002_%_inc,2003_%_inc,2004_%_inc,2005_%_inc,2006_%_inc,2007_%_inc,2008_%_inc,2009_%_inc,2010_%_inc
0,North America,1.302175,1.286250,1.249080,1.201161,1.195849,1.217137,1.212776,1.207640,1.242781,...,1.090635,1.010051,0.950047,0.987414,0.976816,1.002899,1.029479,0.975903,0.929927,1.052665
1,Bermuda,0.328887,0.473502,0.616277,0.612502,0.590868,0.587398,0.566271,0.598276,0.559734,...,0.872185,0.896086,0.903708,0.895615,0.872360,0.804127,0.722456,0.702331,0.667755,0.633844
2,Canada,1.247088,1.212450,1.009448,0.964398,0.933009,1.010732,1.320038,1.299826,1.801463,...,0.891267,0.842089,0.784900,0.769694,0.783015,0.834610,0.855200,0.840237,0.826521,0.813833
3,Greenland,1.633141,1.234568,0.871080,0.997889,0.988030,0.921919,0.857569,1.386322,1.020966,...,0.421867,0.402591,0.313808,0.278067,0.138648,-0.242298,-0.190840,0.052147,0.069493,0.069444
4,Mexico,2.372845,2.389106,2.403627,2.340775,2.247052,2.182181,2.141775,2.070992,1.938108,...,1.321310,1.217785,1.208168,1.197024,1.184561,1.173819,1.164603,1.154094,1.142636,1.130339
5,Saint Pierre and Miquelon,0.333890,0.665557,0.330579,0.658979,0.818331,0.811688,0.644122,0.480000,0.477707,...,-0.624025,-0.627943,-0.631912,-0.635930,-0.800000,-0.806452,-0.813008,-0.819672,-0.826446,-1.000000
6,United States,0.986262,0.958204,0.918367,0.869538,0.890025,0.928486,0.897849,0.912159,0.948981,...,1.031144,0.954937,0.876468,0.936642,0.923886,0.960281,1.000254,0.926769,0.864625,1.050893
7,Central & South America,2.174231,2.178203,2.143741,2.037747,1.990560,2.028082,2.041957,2.022114,1.956682,...,1.372919,1.360134,1.369030,1.374890,1.357210,1.321779,1.285418,1.263179,1.237416,1.153395
8,Antarctica,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Antigua and Barbuda,-0.423049,-0.366247,-3.514189,-1.752514,-0.108578,0.000000,0.031056,0.015523,-0.062083,...,1.831453,1.576958,1.449833,1.416466,1.359272,1.304134,1.323779,1.306484,1.313299,1.307953


In [18]:
# Define the list of years
years = [(str(year)+'_%_inc') for year in range(1981, 2011)]

# Iterate over each year, finding the maximum percent increase and corresponding country
max_values = []
for year in years:
    # Selecting the country and the year column
    year_column = col(year)
    # Finding the maximum value and corresponding country for the current year
    max_value_row = population.select("_c0", year_column).orderBy(year_column.desc()).limit(1).first()
    max_country = max_value_row["_c0"]
    max_percent_increase = max_value_row[year]
    max_values.append((year[:4], max_country, max_percent_increase))

# Create a DataFrame from the collected maximum values
result_q4 = spark.createDataFrame(max_values, schema=["Year", "Country", "Max_Percent_Increase"])
result_q4.toPandas()

24/03/17 16:55:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1980, 1981
 Schema: _c0, 1980, 1981
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv
24/03/17 16:55:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1981, 1982
 Schema: _c0, 1981, 1982
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv
24/03/17 16:55:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1982, 1983
 Schema: _c0, 1982, 1983
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv
24/03/17 16:55:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1983, 1984
 Schema: _c0, 1983, 1984
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv
24/03/17 16:55:56 WARN CSVHeaderChecker: CSV hea

,Year,Country,Max_Percent_Increase
0,1981,Western Sahara,12.133183
1,1982,Western Sahara,11.115105
2,1983,French Guiana,14.285714
3,1984,Qatar,10.964057
4,1985,French Guiana,12.500000
5,1986,Qatar,8.771733
6,1987,French Guiana,11.111111
7,1988,Cayman Islands,11.010421
8,1989,United Arab Emirates,6.119858
9,1990,Djibouti,12.824048


# 5. 20 Points

Dataset: hw1dir

Solve: WordCount

Do word count exercise using pyspark.
Ignore punctuation and normalize to lower case.
Replace characters in NOT in this set: [0-9a-z] with space.

In [19]:
text = spark.read.text("shared/hw2/hw1text")
text.limit(10).toPandas()

,value
0,
1,@@31754341 <h> COVID-19 : 73 Children Grabbed ...
2,@@31754441 <p> ECONOMYNEXT- Sri Lanka has rela...
3,@@31754641 <p> The Colombo Stock Exchange ( CS...
4,@@31754741 <h> Over 80% of US now in lockdown ...
5,@@31755241 <h> The Virus Is a Reminder of Some...
6,@@31755341 <h> Dozens of University of Texas s...
7,"@@31755441 <p> By now , you may have noticed a..."
8,@@31755641 <h> Share this article <p> Covid-19...
9,@@31755741 <h> With much-needed equipment come...


In [20]:
def normalize(text):
    return lower(regexp_replace(text, r'[^0-9a-z]', ' '))

# Apply normalization function to the text column
normalized = text.withColumn("normalized_text", normalize("value"))
normalized.limit(10).toPandas()

,value,normalized_text
0,,
1,@@31754341 <h> COVID-19 : 73 Children Grabbed ...,31754341 h 19 73 hildren rabbed ...
2,@@31754441 <p> ECONOMYNEXT- Sri Lanka has rela...,31754441 p ri anka has rela...
3,@@31754641 <p> The Colombo Stock Exchange ( CS...,31754641 p he olombo tock xchange ...
4,@@31754741 <h> Over 80% of US now in lockdown ...,31754741 h ver 80 of now in lockdown ...
5,@@31755241 <h> The Virus Is a Reminder of Some...,31755241 h he irus s a eminder of ome...
6,@@31755341 <h> Dozens of University of Texas s...,31755341 h ozens of niversity of exas s...
7,"@@31755441 <p> By now , you may have noticed a...",31755441 p y now you may have noticed a...
8,@@31755641 <h> Share this article <p> Covid-19...,31755641 h hare this article p ovid 19...
9,@@31755741 <h> With much-needed equipment come...,31755741 h ith much needed equipment come...


In [21]:
# Split text into words and explode into separate rows
words = normalized.select(explode(split(normalized["normalized_text"], " ")).alias("word"))
words.limit(10).toPandas()

,word
0,
1,
2,
3,31754341
4,
5,h
6,
7,
8,
9,


In [22]:
# Filter out empty strings
filtered_words = words.filter(words.word != '')
filtered_words.limit(10).toPandas()

,word
0,31754341
1,h
2,19
3,73
4,hildren
5,rabbed
6,n
7,akoradi
8,or
9,awking


In [23]:
# Perform word count
result_q5 = filtered_words.groupBy("word").count().orderBy("count", ascending=False)
result_q5.toPandas()

,word,count
0,the,142965
1,to,87873
2,p,78583
3,of,75074
4,and,70933
...,...,...
67108,unclean,1
67109,31563641,1
67110,backseats,1
67111,forevermore,1


# 6. 15 Points

Dataset: internet_archive_scifi_v3.txt

Find the 10 most common bigrams

In [24]:
lines_rdd = sc.textFile("shared/hw2/internet_archive_scifi_v3.txt")

In [25]:
def normalize_and_split(line):
    return re.sub(r'[^a-zA-Z0-9\s]', '', line.lower()).split()

# Split each line into words and normalize
words_rdd = lines_rdd.flatMap(normalize_and_split)
for item in words_rdd.take(10):
    print(item)

march
all
stories
new
and
complete
publisher
editor
if
is


In [26]:
def create_bigrams(words):
    bigrams = []
    prev_word = None
    for word in words:
        if prev_word:
            bigrams.append((prev_word, word))
        prev_word = word
    return bigrams

bigrams_rdd = words_rdd.mapPartitions(create_bigrams)
for item in bigrams_rdd.take(10):
    print(item)

('march', 'all')
('all', 'stories')
('stories', 'new')
('new', 'and')
('and', 'complete')
('complete', 'publisher')
('publisher', 'editor')
('editor', 'if')
('if', 'is')
('is', 'published')


In [27]:
# Count occurrences of each bigram
bigram_counts_rdd = bigrams_rdd.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
for item in bigram_counts_rdd.take(10):
    print(item)

(('staggering', 'premise'), 1)
(('is', 'ever'), 77)
(('checklist', 'of'), 38)
(('would', 'have'), 11321)
(('hundreds', 'of'), 1154)
(('continued', 'conversationally'), 2)
(('miles', 'and'), 190)
(('years', 'next'), 6)
(('head', 'snapped'), 33)
(('mr', 'cordell'), 14)


In [28]:
# Sort the bigrams by their counts in descending order and take the top 10
result_q6 = bigram_counts_rdd.sortBy(lambda x: x[1], ascending=False).take(10)
result_q6

[(('of', 'the'), 157749),
 (('in', 'the'), 102032),
 (('to', 'the'), 69796),
 (('on', 'the'), 57880),
 (('it', 'was'), 56131),
 (('and', 'the'), 46022),
 (('at', 'the'), 43653),
 (('to', 'be'), 39357),
 (('he', 'was'), 38249),
 (('he', 'had'), 36679)]

# 7. Extra credit – 25 points

Datasets: durham-nc-foreclosure-2006-2016.json, Restaurants_in_Durham_County_NC.json

Solve: For each restaurant (‘Restaurants_in_Durham_County_NC.json’), “status” = “ACTIVE” and “"rpt_area_desc" = "Food Service":

Show the number of foreclosures (‘durham-nc-foreclosure-2006-2016’) within 3 miles of the restaurant’s coordinates.

You can use the haversine library/function from here: https://pypi.org/project/haversine/

In [29]:
# Filter out restaurants
restaurants_filtered = restaurants.filter((col("status") == "ACTIVE") & (col("rpt_area_desc") == "Food Service"))
restaurants_filtered.limit(10).toPandas()

,closing_date,est_group_desc,geolocation,hours_of_operation,id,insp_freq,opening_date,premise_address1,premise_address2,premise_city,...,premise_zip,risk,rpt_area_desc,seats,sewage,smoking_allowed,status,transitional_type_desc,type_description,water
0,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,27707,4,Food Service,60,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community
1,None,Nursing Home,"[36.0467802, -78.8895483]",None,58123,4,2003-10-15,4434 BEN FRANKLIN BLVD,None,DURHAM,...,27704,4,Food Service,350,3 - Municipal/Community,NO,ACTIVE,FOOD,16 - Institutional Food Service,5 - Municipal/Community
2,None,Fast Food Restaurant,"[35.9182655, -78.9593263]",None,70266,2,2009-07-09,1125 W. NC HWY 54 SUITE 806,None,DURHAM,...,27707,2,Food Service,7,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community
3,None,Full-Service Restaurant,"[36.0183378, -78.9060312]",None,97837,2,2012-01-09,1542 N GREGSON ST,None,DURHAM,...,27701,2,Food Service,100,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community
4,None,Meat and Poultry Department,"[35.9495321, -78.9211694]",None,58349,3,2003-03-03,3825 S ROXBORO ST,SUITE 101,DURHAM,...,27707,3,Food Service,0,3 - Municipal/Community,N/A,ACTIVE,FOOD,30 - Meat Market,5 - Municipal/Community
5,None,None,"[35.9852771, -78.898944]",None,58342,3,2001-08-17,810 FAYETTEVILLE ST,SUITE 108,DURHAM,...,27701,3,Food Service,0,3 - Municipal/Community,NO,ACTIVE,FOOD,30 - Meat Market,5 - Municipal/Community
6,None,Fast Food Restaurant,"[36.0586094, -78.9279599]",None,57278,2,1990-07-01,3808 GUESS ROAD,None,DURHAM,...,27705,2,Food Service,0,3 - Municipal/Community,N/A,ACTIVE,FOOD,2 - Food Stands,5 - Municipal/Community
7,None,Fast Food Restaurant,"[36.0094173, -78.9218729]",None,57190,4,2008-04-18,748 NINTH STREET,None,DURHAM,...,27705,4,Food Service,36,3 - Municipal/Community,N/A,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community
8,None,Fast Food Restaurant,"[36.018111, -78.9102149]",None,57585,2,1999-01-19,1058 W CLUB,None,DURHAM,...,27701,2,Food Service,0,3 - Municipal/Community,NO,ACTIVE,FOOD,2 - Food Stands,5 - Municipal/Community
9,None,Fast Food Restaurant,"[36.001517, -78.9391734]",None,178137,4,2016-07-11,406 CHAPEL DRIVE,PO BOX 90898,DURHAM,...,27708,4,Food Service,100,3 - Municipal/Community,N/A,ACTIVE,FOOD,6 - Edu. Food Service,5 - Municipal/Community


In [30]:
foreclosure = spark.read.json("shared/hw2/durham-nc-foreclosure-2006-2016.json")
foreclosure = foreclosure.select(col("fields.*"))
foreclosure.limit(10).toPandas()

,address,geocode,parcel_number,year
0,217 E CORPORATION ST,"[36.0013755, -78.8922549]",110138,2006
1,401 N QUEEN ST,"[35.995797, -78.895396]",110535,2006
2,403 N QUEEN ST,"[35.995413, -78.8950321]",110536,2006
3,918 GILBERT ST,"[35.9957683, -78.8873774]",111324,2006
4,721 LIBERTY ST,"[35.993026, -78.888343]",111399,2006
5,729 HOPKINS ST,"[35.99217, -78.888092]",111426,2006
6,1302 E MAIN ST,"[35.9865799, -78.886681]",112166,2006
7,402 CLAY ST,"[35.9815968, -78.8806365]",112278,2006
8,1516 LATHROP ST,"[35.992438, -78.874621]",112934,2006
9,2604 E MAIN ST,"[35.985903, -78.869642]",114277,2006


In [31]:
# Cross join the dataframes
combined = restaurants_filtered.crossJoin(foreclosure)
# Filter out rows with missing or invalid geolocation or geocode coordinates
combined = combined.filter((col("geolocation").isNotNull()) & (col("geocode").isNotNull()))
combined.limit(10).toPandas()

,closing_date,est_group_desc,geolocation,hours_of_operation,id,insp_freq,opening_date,premise_address1,premise_address2,premise_city,...,sewage,smoking_allowed,status,transitional_type_desc,type_description,water,address,geocode,parcel_number,year
0,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,217 E CORPORATION ST,"[36.0013755, -78.8922549]",110138,2006
1,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,401 N QUEEN ST,"[35.995797, -78.895396]",110535,2006
2,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,403 N QUEEN ST,"[35.995413, -78.8950321]",110536,2006
3,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,918 GILBERT ST,"[35.9957683, -78.8873774]",111324,2006
4,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,721 LIBERTY ST,"[35.993026, -78.888343]",111399,2006
5,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,729 HOPKINS ST,"[35.99217, -78.888092]",111426,2006
6,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,1302 E MAIN ST,"[35.9865799, -78.886681]",112166,2006
7,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,402 CLAY ST,"[35.9815968, -78.8806365]",112278,2006
8,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,1516 LATHROP ST,"[35.992438, -78.874621]",112934,2006
9,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,3 - Municipal/Community,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,2604 E MAIN ST,"[35.985903, -78.869642]",114277,2006


In [32]:
def haversine(lat1, lon1, lat2, lon2):
    R = 3958.8  # Radius of the Earth in miles
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Define a UDF from the haversine function
haversine_udf = udf(haversine, FloatType())

# Now you can use the UDF with withColumn
combined = combined.withColumn("distance", haversine_udf(combined["geolocation"][0], combined["geolocation"][1], combined["geocode"][0], combined["geocode"][1]))
combined.limit(10).toPandas()

,closing_date,est_group_desc,geolocation,hours_of_operation,id,insp_freq,opening_date,premise_address1,premise_address2,premise_city,...,smoking_allowed,status,transitional_type_desc,type_description,water,address,geocode,parcel_number,year,distance
0,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,217 E CORPORATION ST,"[36.0013755, -78.8922549]",110138,2006,6.655511
1,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,401 N QUEEN ST,"[35.995797, -78.895396]",110535,2006,6.237134
2,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,403 N QUEEN ST,"[35.995413, -78.8950321]",110536,2006,6.226458
3,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,918 GILBERT ST,"[35.9957683, -78.8873774]",111324,2006,6.495321
4,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,721 LIBERTY ST,"[35.993026, -78.888343]",111399,2006,6.311984
5,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,729 HOPKINS ST,"[35.99217, -78.888092]",111426,2006,6.273948
6,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,1302 E MAIN ST,"[35.9865799, -78.886681]",112166,2006,6.026353
7,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,402 CLAY ST,"[35.9815968, -78.8806365]",112278,2006,6.007457
8,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,1516 LATHROP ST,"[35.992438, -78.874621]",112934,2006,6.778515
9,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,2604 E MAIN ST,"[35.985903, -78.869642]",114277,2006,6.658343


In [33]:
within_3_miles = combined.filter(col("distance") <= 3.0)
within_3_miles.limit(10).toPandas()

24/03/17 17:00:03 WARN ExtractPythonUDFFromJoinCondition: The join condition:(haversine(geolocation#184[0], geolocation#184[1], geocode#2254[0], geocode#2254[1])#2291 <= 3.0) of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.


,closing_date,est_group_desc,geolocation,hours_of_operation,id,insp_freq,opening_date,premise_address1,premise_address2,premise_city,...,smoking_allowed,status,transitional_type_desc,type_description,water,address,geocode,parcel_number,year,distance
0,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,801 BROOKHAVEN DR,"[35.952835, -78.940042]",122866,2006,2.420107
1,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,805 BROOKHAVEN DR,"[35.952647, -78.9402719]",122867,2006,2.403069
2,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,7 WEYBRIDGE PL,"[35.9079369, -78.987854]",142559,2006,1.923169
3,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,7412 STAR DR,"[35.888512, -78.979971]",143262,2006,2.561280
4,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,PANAMA TER,"[35.9364633, -78.9130833]",146995,2006,2.703792
5,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,700 BROOKHAVEN DR,"[35.953179, -78.93967]",122865,2007,2.450229
6,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,1021 STONEHEDGE AVE,"[35.952122, -78.945094]",122760,2006,2.274637
7,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,1124 SAXONY DR,"[35.954764, -78.946299]",122819,2006,2.431357
8,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,700 BROOKHAVEN DR,"[35.953179, -78.93967]",122865,2006,2.450229
9,None,Full-Service Restaurant,"[35.9207272, -78.9573299]",None,56060,4,1994-09-01,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,...,NO,ACTIVE,FOOD,1 - Restaurant,5 - Municipal/Community,600 REYNOLDS AVE,"[35.953408, -78.935073]",134703,2006,2.578585


In [34]:
result_q7 = within_3_miles.groupBy("premise_name").count()
result_q7.limit(10).toPandas()

24/03/17 17:00:06 WARN ExtractPythonUDFFromJoinCondition: The join condition:(haversine(geolocation#184[0], geolocation#184[1], geocode#2254[0], geocode#2254[1])#2291 <= 3.0) of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.


,premise_name,count
0,GRILL 46,79
1,EL DORADO'S #6,57
2,DUKE UNIVERSITY WEST UNION TANDOOR,471
3,COMPARE FOODS DELI,1122
4,BLU SEAFOOD AND BAR,755
5,MCDONALD'S 35265,172
6,GSK COMMERCIAL OPERATIONS CENTER,137
7,JADE BUFFET,789
8,W G PEARSON SCHOOL LUNCHROOM,955
9,DPAC 3RD FLOOR,1286
